# Problem Statement

The objective of this competition is to predict the probability that a customer does not pay back their credit card balance amount in the future based on their monthly customer profile. The target binary variable is calculated by observing 18 months performance window after the latest credit card statement, and if the customer does not pay due amount in 120 days after their latest statement date it is considered a default event.

The dataset contains aggregated profile features for each customer at each statement date. Features are anonymized and normalized, and fall into the following general categories:

- D_* = Delinquency variables
- S_* = Spend variables
- P_* = Payment variables
- B_* = Balance variables
- R_* = Risk variables

With the following features being categorical:

['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

Our task is to predict, for each customer_ID, the probability of a future payment default (target = 1).

Note that the negative class has been subsampled for this dataset at 5%, and thus receives a 20x weighting in the scoring metric.

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Filtering the Large Dataset

In [3]:
# Chunk = pd.read_csv("train_data.csv",chunksize=100000)
# Train = pd.concat(Chunk)
# Train_labels = pd.read_csv("train_labels.csv")

# Train = pd.merge(Train,Train_labels,how="inner",on=["customer_ID"])

# Train = Train.head(200000)
# Train.to_csv("Train.csv",index=False)

# Test = pd.read_csv("test_data.csv",chunksize=100000)
# Test = pd.concat(Test)
# Test = Test.head(500000)
# Test.to_csv("Test.csv",index = False)

In [9]:
Train = pd.read_csv("Train.csv")
Test = pd.read_csv("Test.csv")

In [11]:
Test.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-02-19,0.631315,0.001912,0.010728,0.814497,0.007547,0.168651,0.009971,0.002347,...,NaN,NaN,NaN,NaN,0.004669,NaN,NaN,NaN,0.008281,NaN
1,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-03-25,0.587042,0.005275,0.011026,0.810848,0.001817,0.241389,0.000166,0.009132,...,NaN,NaN,NaN,0.000142,0.004940,0.009021,NaN,0.003695,0.003753,0.001460
2,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-04-25,0.609056,0.003326,0.016390,1.004620,0.000114,0.266976,0.004196,0.004192,...,NaN,NaN,NaN,0.000074,0.002114,0.004656,NaN,0.003155,0.002156,0.006482
3,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-05-20,0.614911,0.009065,0.021672,0.816549,0.009722,0.188947,0.004123,0.015325,...,NaN,NaN,NaN,0.004743,0.006392,0.002890,NaN,0.006044,0.005206,0.007855
4,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-06-15,0.591673,0.238794,0.015923,0.810456,0.002026,0.180035,0.000731,0.011281,...,NaN,NaN,NaN,0.008133,0.004329,0.008384,NaN,0.001008,0.007421,0.009471


In [13]:
Train.nunique()

customer_ID     16576
S_2               396
P_2            198458
D_39           200000
B_1            200000
                ...  
D_142           34356
D_143          196497
D_144          198567
D_145          196497
target              2
Length: 191, dtype: int64

In [14]:
Train.describe()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
count,198458.000000,2.000000e+05,200000.000000,199928.000000,2.000000e+05,162335.000000,1.999280e+05,1.999280e+05,29193.000000,1.399420e+05,...,7.186000e+03,7186.000000,1.964970e+05,1.985810e+05,1.964970e+05,34356.000000,1.964970e+05,1.985670e+05,1.964970e+05,200000.000000
mean,0.653527,1.534126e-01,0.125356,0.616971,8.121541e-02,0.227386,6.221769e-02,1.343925e-01,0.181170,1.560354e-01,...,1.424359e-02,0.162349,1.798358e-01,2.622542e-02,1.653822e-01,0.392596,1.797362e-01,5.339187e-02,6.179908e-02,0.253395
std,0.246135,2.723813e-01,0.213396,0.402275,2.309947e-01,0.197101,2.081621e-01,2.348492e-01,0.216851,2.169740e-01,...,9.543979e-02,0.258647,3.798505e-01,1.441568e-01,3.490555e-01,0.238923,3.797464e-01,1.851044e-01,1.901802e-01,0.434956
min,-0.383019,3.892609e-07,-0.899396,0.000003,2.962930e-08,-0.254707,5.627163e-08,1.042180e-07,-0.000219,8.705647e-07,...,4.129697e-08,0.000002,7.139375e-08,5.277736e-08,5.642931e-08,-0.014441,1.653580e-08,1.161969e-07,3.397747e-08,0.000000
25%,0.476334,4.541390e-03,0.008858,0.100550,2.899612e-03,0.127432,2.897833e-03,5.290730e-03,0.039449,4.235285e-02,...,2.589397e-03,0.003541,3.018427e-03,2.551375e-03,3.033245e-03,0.196174,3.034162e-03,2.752575e-03,3.027842e-03,0.000000
50%,0.691541,9.066380e-03,0.031935,0.814164,5.792558e-03,0.164248,5.768750e-03,9.872834e-03,0.120749,8.828850e-02,...,5.141117e-03,0.007028,6.041458e-03,5.109293e-03,6.062756e-03,0.382038,6.077791e-03,5.499113e-03,6.059988e-03,0.000000
75%,0.863455,2.360171e-01,0.129569,1.002240,8.684554e-03,0.260202,8.652639e-03,1.635911e-01,0.250728,1.850985e-01,...,7.667529e-03,0.501621,9.096570e-03,7.658175e-03,9.094362e-03,0.566102,9.088631e-03,8.267479e-03,9.093526e-03,1.000000
max,1.009998,5.331360e+00,1.324053,1.010000,2.507711e+00,2.918675,6.798167e+00,1.625262e+00,3.252056,9.089694e+00,...,1.009913e+00,1.509486,1.010000e+00,1.009994e+00,1.174753e+00,1.751388,1.010000e+00,1.343284e+00,4.282032e+00,1.000000


In [21]:
print(Train.isnull().sum())

customer_ID         0
S_2                 0
P_2              1542
D_39                0
B_1                 0
                ...  
D_142          165644
D_143            3503
D_144            1433
D_145            3503
target              0
Length: 191, dtype: int64


In [41]:
Categorical_Features = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

for features in Categorical_Features:
    print(Train[features].unique())
    print(Train[features].value_counts())
    

[ 0.  2.  1. nan]
B_30
0.0    169651
1.0     28195
2.0      2082
Name: count, dtype: int64
[ 2.  1.  3.  5.  6.  7.  4. nan]
B_38
2.0    69185
3.0    45557
1.0    42290
5.0    16264
4.0    10948
7.0     9821
6.0     5863
Name: count, dtype: int64
[ 1.  0. nan]
D_114
1.0    119921
0.0     73734
Name: count, dtype: int64
[ 0. nan  1.]
D_116
0.0    193429
1.0       226
Name: count, dtype: int64
[ 4. -1.  6.  2.  1. nan  3.  5.]
D_117
-1.0    52208
 3.0    41808
 4.0    41322
 2.0    24095
 5.0    17049
 6.0    12774
 1.0     4399
Name: count, dtype: int64
[ 0.  1. nan]
D_120
0.0    170250
1.0     23405
Name: count, dtype: int64
[ 1. nan  0. -1.]
D_126
 1.0    154085
 0.0     32515
-1.0      9175
Name: count, dtype: int64
['CR' 'CO' 'CL' 'XZ' 'XM' 'XL']
D_63
CO    148773
CR     33751
CL     15991
XZ       925
XM       309
XL       251
Name: count, dtype: int64
['O' 'R' nan 'U' '-1']
D_64
O     105870
U      55177
R      29601
-1      1312
Name: count, dtype: int64
[nan  1.  0.]
D_66
1.0   